# Principal Component Analysis (PCA)


In [ ]:
import pandas as pd
import numpy as np
import altair as alt

This example is adopted from the article [PCA Example in Python with scikit-learn](https://cmdlinetips.com/2018/03/pca-example-in-python-with-scikit-learn/).

In [ ]:
# https://cmdlinetips.com/2018/03/pca-example-in-python-with-scikit-learn/
# load make_blobs to simulate data
from sklearn.datasets import make_blobs
# load decomposition to do PCA analysis with sklearn
from sklearn import decomposition

`make_blobs` is one of the modules available in `scikit-learn` to construct simulated data. `make_blobs` can be easily used to make data set with multiple gaussian clusters and is widely used to test clustering algorithms. 

In [ ]:
help(make_blobs)

Here we will use `make_blobs` to generate 100 x 10 matrix data, such that there are 100 samples with 10 observations (`n_features=10`). These 100 samples were generated from four different clusters (`centers=4`). Since it is simulated, we know which cluster each sample belong to (i.e., "cluster assignment").

In [ ]:
X1, Y1 = make_blobs(n_features=10, 
         n_samples=100,
         centers=4, random_state=4,
         cluster_std=2)
print(X1.shape)

Here, `X1` is the 100 x 10 data and `Y1` is cluster assignment for the 100 samples.

In [ ]:
df = pd.DataFrame({
  'Y': Y1
})

In [ ]:
df = pd.concat([df, pd.DataFrame(X1)], axis=1)

In [ ]:
df

In [ ]:
# Rename the columns to be able to plot it in Altair
# WARNING: running this again will rename the columns again!
col_names = list(df.columns)
df_renamed = df
for col in col_names:
    df_renamed = df_renamed.rename(columns={col : "F"+str(col)})
    
df_renamed.head()

In [ ]:
## alt.Chart(df).mark_circle().encode(
##     x = "1",
##     y = "0"
## )
## The code above wasn't working with the original dataframe
## which is why we renamed the columns.
## Now, we can plot columns and visualize them against one another
## Try changing the column names to see how the relationship changes.
alt.Chart(df_renamed).mark_circle().encode(
    x = "F3",
    y = "F1"
)

The code below is just an alternative way to visualize the range of values for each feature.

In [ ]:
df1 = df.melt('Y')
df1

In [ ]:
alt.Chart(df1).mark_point().encode(
    alt.X('variable'),
    alt.Y('value')
)

In [ ]:
alt.Chart(df1).mark_point().encode(
    alt.X('variable'),
    alt.Y('value'),
    color='Y:N'
)

~~The simulated data is already centered and scaled, so we can go ahead and fit PCA model.~~

**IMPORTANT**: Read more about the [Importance of Feature Scaling](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#sphx-glr-auto-examples-preprocessing-plot-scaling-importance-py).


    "Feature scaling through standardization (or Z-score normalization) can be an important preprocessing step for many machine learning algorithms. Standardization involves rescaling the features such that they have the properties of a standard normal distribution with a mean of zero and a standard deviation of one.

    ... In PCA we are interested in the components that maximize the variance. If one component (e.g. human height) varies less than another (e.g. weight) because of their respective scales (meters vs. kilos), PCA might determine that the direction of maximal variance more closely corresponds with the ‘weight’ axis, if those features are not scaled. As a change in height of one meter can be considered much more important than the change in weight of one kilogram, this is clearly incorrect."

In [ ]:
X1.mean()

In [ ]:
norm_X1 = X1 - np.mean(X1)
norm_X1.mean()

In [ ]:
# ##  
# np.mean(df)
# df - np.mean(df)
#df

In [ ]:
pca = decomposition.PCA(n_components=4)

We will fit PCA model using `fit_transform function` to our data X1 and the result `pc` contains the **principal components**.


In [ ]:
pc = pca.fit_transform(X1)
pc.shape

### Aside: manually computing SVD

PCA is a specific application of the singular value decomposition (SVD) for matrices. If we have a data matrix $X$, we can decompose it into $U$, $\Sigma$ and $V^T$ such that $$X = U \Sigma V^T.$$ Here, $U$ is the left singular vectors, $\Sigma$ is a diagonal matrix containing the singular values, and $V^T$ are the right singular vectors.


$U$ is orthonormal, so $U^T U = I$

and

$V^T$ is orthonormal, so $V^T V = I$.

Re-writing the above equation (by right-multiplying by $V$) gives us $XV = U \Sigma V^T V$ resulting in 
$$XV = U \Sigma,$$ where $U \Sigma$ is our reduced data.

Our original data matrix $X$ has $n$ observations and $p$ variabes (features), so its dimensions are $n \times p$. If we want to reduce our original $p$ features down to $r$ features ($r$ much less than $p$), our matrices need to be as follows:

$X$ is $n \times p$

$U$ is $n \times r$ (orthonormal)

$\Sigma$ is $r \times r$ (diagonal matrix)

$V^T$ is $r \times p$ (orthonormal)

In the following cell, use the [`np.linalg.svd`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.svd.html) function to compute the SVD of `X1`. We will store the left singular vectors, singular values, and right singular vectors in `u`, `s`, and `vt` respectively. 

In [ ]:
#norm_X1 = pd.DataFrame(X1).sub(pd.DataFrame(X1).mean(axis=0), axis=1)
u, s, vt = np.linalg.svd(norm_X1, full_matrices = False)
#print("new mean", norm_X1.mean())
print("X:", X1.shape)
print("u:", u.shape)
print("s:", s.shape)
print("vt:", vt.shape)

Let's now consider the relationship between the singular values `s` and the variance of our data. **The total variance** is the **sum of the variances of each column** of our data. Let's compute the variance for each column of the data.

In [ ]:
#help(np.var)

In [ ]:
print("Total var =", sum(np.var(X1, axis=0)))
np.var(X1, axis=0)

In [ ]:
print("Total var =", sum(np.var(norm_X1, axis=0)))
np.var(norm_X1, axis=0)

**The total variance** of the data is also equal to the **sum of the squares of the singular values divided by the number of data points**, that is:

$$Var(X) = \frac{\sum_{i=1}^k{s_i^2}}{N}$$

In [ ]:
N = len(X1)
total_variance_computed_from_singular_values = np.sum(np.square(s)) / N
total_variance_computed_from_singular_values

The amount of variance captured by the $i$th principal component is equal to ($i$th singular value)$^2 / \sum_{i=1}^k{s_i^2}$

In [ ]:
variance_of_1st_pc = s[0]**2 / sum(s**2)
variance_of_1st_pc

### End of Aside: check what sklearn.pca produced

Let us make a pandas data frame with the principal components (PCs) and add the known cluster assignments.

In [ ]:
pc_df = pd.DataFrame(data = pc, 
        columns = ['PC1', 'PC2','PC3','PC4'])
pc_df['Cluster'] = Y1 # create a new column for clusters

print(pc_df.shape)
pc_df.head()

Let us examine the variance explained by each principal component. We can see that the first two principal components explains over 70% of the variation in the data.

In [ ]:
pca.explained_variance_ratio_

Let us plot the variance explained by each principal component. This is also called **Scree plot**.

In [ ]:
df2 = pd.DataFrame({
    'var':pca.explained_variance_ratio_,
    'PC':['PC1','PC2','PC3','PC4']}
)
df2

In [ ]:
alt.Chart(df2, title="Variance Explained by Principal Components").mark_bar().encode(
    alt.X('PC'),
    alt.Y('var')
)

Now we can use the top two principal components and make scatter plot. 

In [ ]:
pc_df2 = pc_df.drop(["PC3", "PC4"], axis=1)
pc_df2.head()

In [ ]:
alt.Chart(pc_df2).mark_point().encode(
    alt.X('PC1:Q'),
    alt.Y('PC2:Q'),
    #color='Cluster:N'
)


We can clearly see the four clusters in our data. The two principal components are able to completely separate the clusters.
